<a href="https://colab.research.google.com/github/SYEON9/natural_language_3th/blob/main/HW/%5BHW17%5DLR_vs_MLP_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LR vs MLP

이전 시간까지는 layer를 하나만 쌓아서 학습했다. 이번에는 layer 개수에 따라 성능이 어떻게 바뀌는지에 대해서 알아보자. 

layer를 하나 쌓아서 학습하는 linear regression과 non-linear를 추가해서 layer 2개를 쌍항서 만든 MLP를 비교해보자. 

데이터는 metal-casting parts dataset을 활용한다. 

#LR vs MLP
이전 시간까지는 layer를 하나만 쌓아서 학습을 했다. 이번 시간에는 layer의 개수에 따라서 성능이 어떻게 바뀌는지에 대해서 알아보도록 하자.

Layer를 하나 쌓아서 학습하는 Linear regression과 non-linear 를 추가해서 layer 2개를 쌓아서 만든 MLP를 비교해보자. 

데이터는 metal-casting parts dataset을 활용한다.



#images of metal-casting parts

In [ ]:
from autograd import numpy
from autograd import grad
from matplotlib import pyplot

In [ ]:
from urllib.request import urlretrieve
URL = 'https://github.com/engineersCode/EngComp6_deeplearning/raw/master/data/casting_images.npz'
urlretrieve(URL, 'casting_images.npz')

In [ ]:
#read in images and labels
with numpy.load('/content/casting_images.npz', allow_pickle = True) as data:
    ok_images = data['ok_images']
    def_images = data['def_images']

In [ ]:
type(ok_images)

In [ ]:
ok_images.dtype

In [ ]:
ok_images.shape

In [ ]:
def_images.shape

[HW14]에서 했던 것과 동일하게 진행한다.

In [ ]:
n_ok_total = ok_images.shape[0]
res = int(numpy.sqrt(def_images.shape[1]))   #이미지의 가로/세로 크기 찾기

print("Number of images without defects: ", n_ok_total)
print("Image resolution: {} by {}".format(res, res))

In [ ]:
n_def_total = def_images.shape[0]
print("Number of images with defects: ", n_def_total)

In [ ]:
fig, axes = pyplot.subplots(2,3, figsize = (8,6), tight_layout = True)
axes[0,0].imshow(ok_images[0].reshape((res,res)), cmap = 'gray')
axes[0,1].imshow(ok_images[50].reshape((res,res)), cmap = 'gray')
axes[0, 2].imshow(ok_images[100].reshape((res, res)), cmap="gray")
axes[1, 0].imshow(ok_images[150].reshape((res, res)), cmap="gray")
axes[1, 1].imshow(ok_images[200].reshape((res, res)), cmap="gray")
axes[1, 2].imshow(ok_images[250].reshape((res, res)), cmap="gray")
fig.suptitle('Casting parts without defects', fontsize = 20);

In [ ]:
fig, axes = pyplot.subplots(2,3,figsize = (8,6), tight_layout = True)
axes[0,0].imshow(def_images[0].reshape((res,res)), cmap = 'gray')
axes[0, 1].imshow(def_images[50].reshape((res, res)), cmap="gray")
axes[0, 2].imshow(def_images[100].reshape((res, res)), cmap="gray")
axes[1, 0].imshow(def_images[150].reshape((res, res)), cmap="gray")
axes[1, 1].imshow(def_images[200].reshape((res, res)), cmap="gray")
axes[1, 2].imshow(def_images[250].reshape((res, res)), cmap="gray")
fig.suptitle("Casting parts with defects", fontsize = 20);

#Split Dataset

학습과 평가를 위한 dataset으로 나눈다. train, val, test 데이터에 대한 각각의 개수를 구한다. 

In [ ]:
#number of images for validation(~20%)
n_ok_val = int(n_ok_total * 0.2)
n_def_val = int(n_def_total * 0.2)
print("Number of images without defects in validation dataset:", n_ok_val)
print("Number of images with defects in validation dataset: ",n_def_val)

In [ ]:
#number of images for test(~20%)
n_ok_test = int(n_ok_total * 0.2)
n_def_test = int(n_def_total * 0.2)
print("Number of images without defects in test dataset", n_ok_test)
print("Number of images with defects in test dataset", n_def_test)

In [ ]:
#number of images for train(~60%)
n_ok_train = n_ok_total - n_ok_val - n_ok_test
n_def_train = n_def_total - n_def_val - n_def_test
print("Number of images without defects in train dataset", n_ok_train)
print("Number of images with defects in train dataset", n_def_train)

이제 numpy 패키지 않에 split 함수로 나누어준다.

In [ ]:
ok_images = numpy.split(ok_images, [n_ok_val, n_ok_val+n_ok_test], 0)
def_images = numpy.split(def_images, [n_def_val, n_def_val+n_def_test], 0)

In [ ]:
ok_images

이제 numpy 패키지 안에 concatenate함수를 이용해서 train, val, test끼리 결함이 있는 이미지와 없는 이미지를 합쳐준다.

In [ ]:
images_val = numpy.concatenate([ok_images[0], def_images[0]], 0)
images_test = numpy.concatenate([ok_images[1], def_images[1]], 0)
images_train = numpy.concatenate([ok_images[2], def_images[2]], 0)

#Data normalization: z-score normalization

[HW14]와 동일한 방식으로 진행한다. 
z = (x-train의 평균) / (train의 표준편차)

train, validation, test 모두에 적용한다. 

In [ ]:
images_train.max(), images_train.min()

In [ ]:
#calculate mu and sigma
mu = numpy.mean(images_train, axis = 0)
sigma = numpy.std(images_train, axis = 0)

#normalization the training, validation, test dataset
images_train = (images_train - mu) / sigma
images_val = (images_val - mu) / sigma
images_test = (images_test - mu) / sigma

In [ ]:
images_train.max(), images_train.min()

#Creating labels / classes

데이터셋에 class label을 정해주어야 한다. 이 이미지가 결함이 있느지 없는지 명시적으로 나타내주는 것이다. 결함이 있는 것을 1, 없는 것을 0으로 나타내겠다.

In [ ]:
#label은 정답으로 원래 데이터에 붙일 필요가 없다. 그러므로 정답 벡터만 만들자!

#labels for training data
labels_train = numpy.zeros(n_ok_train+n_def_train)
labels_train[n_ok_train:] = 1.

#labels for validation data
labels_val = numpy.zeros(n_ok_val + n_def_val)
labels_val[n_ok_val:] = 1.

#labels for test data
labels_test = numpy.zeros(n_ok_test + n_def_test)
labels_test[n_ok_test:] = 1.

In [ ]:
labels_test

이제 입력으로 들어온 이미지에 결함이 없는지 알아내기 위해 logistic model을 사용하겠다. 지난번처럼 출력 확률 >= 0.5이면 결함이 있고 출력 확률 < 0.5이면 결함이 없다고 설정한다. 

In [ ]:
def classify(x, model, params):

    probabilites = model(x, params)
    labels = (probabilites >= 0.5).astype(float)
    return labels

#Evaluating model performance: F-score, Accuracy

이제 우리가 학습한 모델이 얼마나 잘 예측하는지 알아보자. 

우리가 사용할 결과지표는 accuarcy, precision, recall이다. 우리는 precision과 recall로 F-score를 계산할 수 있다. 


In [ ]:
def performance(predictions, answers, beta = 1.0):

    true_idx = (answers == 1)    #the location where the answers are 1
    false_idx = (answers == 0)   #the location where the answers are0

    #true positive: answers are 1 and predictions are also 1
    n_tp = numpy.count_nonzero(predictions[true_idx] == 1)

    #false positive: answers are 0 but predictions are 1
    n_fp = numpy.count_nonzero(predictions[false_idx]==1)

    #true negative: answers are 0 and predictions are also 0
    n_tn = numpy.count_nonzero(predictions[false_idx] == 0)

    #false negative: answers are 0 and predictions are 1
    n_fn = numpy.count_nonzero(predictions[true_idx]==0)


    #precision, recall, and f-score
    precision = n_tp/ (n_tp+ n_fp)
    recall = n_tp / (n_tp + n_fn)
    score = (
        (1.0 + beta**2) * precision * recall / 
        (beta**2 * precision + recall)
    )

    accuracy = (n_tp + n_tn) / (n_tp + n_fn + n_fp + n_tn)

    return precision, recall, score, accuracy


#Model

logistic regression과 2 layers MLP model을 구현해보겠다. 

In [ ]:
def logistic(x):
    return 1. / (1. + numpy.exp(-x))

In [ ]:
def LR_model(x, params):
    """A logistic regression model.

    A logistic regression is y = sigmoid(x*w + b), where the operator * denotes
    a mat-vec multiplication.
    """

    return logistic(numpy.dot(x, params[0])+ params[1])

In [ ]:
def MLP_model(x, params):
    """A MLP model.

    A MLP is y = sigmoid(max((x*w + b1), 0) *w2 + b2), 
    where the operator * denotes a mat-vec multiplication.
    """

    x = numpy.dot(x, params[0]) + params[1]
    x = numpy.maximum(x, 0)

    return logistic(numpy.dot(x, params[2]) + params[3])

이제 cost function을 만들어보자. logistic regression의 cost function을 사용하겠다. 

In [ ]:
def model_loss(x, true_labels, model, params):
    
    pred = model(x, params)

    loss = -(
        numpy.dot(true_labels, numpy.log(pred + 1e-15)) +
        numpy.dot(1. - true_labels, numpy.log(1. -pred + 1e-15))
    )

    return loss

In [ ]:
images_train.shape[1]

Initalization

LR과 MLP에서 사용할 parameter들을 초기화하자.

In [ ]:
# a function to get the gradients of a logistic model
gradients = grad(model_loss, argnum = 3)     #output의 gradient 합계를 계산하고 반환한다. 

#initialize LR parameters
std = 1e-4
LR_w = std * numpy.random.randn(images_train.shape[1])
LR_b = numpy.zeros(1)


#initalization MLP parameters
hidden = 32
w0 = std * numpy.random.randn(images_train.shape[1], hidden)
b0 = numpy.zeros(hidden)
w1 = std * numpy.random.rand(hidden)
b1 = numpy.zeros(1)

#training / optimization

이제 두 모델을 학습하고, 성능을 비교해보자.

총 5000번 학습을 하는 동안 가장 높은 accuracy를 측정하겠다.

#logistic regression

In [ ]:
#learning rate
lr = 1e-5

# a variable for optimization iterations
change = numpy.inf

#a counter for optimization iterations
i = 0

# a variable to store the validation loss from the previous iteration
old_val_loss = 1e-15
best_acc = 0.0

#keep running if:
#   1.we still see significant changes in validation loss
#   2. iteration counter < 10000

while i < 5000:

    #학습
    #calculate gradients and use gradient descents
    grads = gradients(images_train, labels_train, LR_model, (LR_w, LR_b))
    LR_w -= (grads[0] * lr)
    LR_b -= (grads[1] * lr)


    #검즘
    #validation loss
    val_loss = model_loss(images_val, labels_val, LR_model, (LR_w, LR_b))

    #calculate f-scores against the validation dataset
    pred_labels_val = classify(images_val, LR_model, (LR_w, LR_b))
    score = performance(pred_labels_val, labels_val)
    best_acc = max(best_acc, score[3])
    #calculate the change in validation loss
    change = numpy.abs((val_loss - old_val_loss)/old_val_loss)

    #update the counter and old_val_loss
    i += 1
    old_val_loss = val_loss

    # print the progress every 10 steps
    if i % 10 == 0:
        print("{}...".format(i), end = '')
    
score = performance(pred_labels_val, labels_val)
print("")
print("")
print("Upon optimization stopped:")
print("    Iterations:", i)
print("    Best Accuracy:", best_acc)

#MLP

In [ ]:
#learning rate
lr = 1e-15

#a variable for the change in validation loss
change = numpy.inf

# a counter for optimization iterations
i = 0

# a variables to store the validation loss from the previous iteration
old_val_loss = 1e-15
best_acc = 0.0


#keep running if:
#   1. we still see significant changes in validation loss
#   2. iteration counter < 10000

while i < 5000:

    #학습
    #calculate gradients and use gradient descents
    grads = gradients(images_train, labels_train, MLP_model, (w0, b0, w1, b1))
    w0 -= (grads[0] * lr)
    b0 -= (grads[1] * lr)
    w1 -= (grads[2] * lr)
    b1 -= (grads[3] * lr)

    
    #검증
    #validation loss
    val_loss = model_loss(images_val, labels_val, MLP_model, (w0, b0, w1, b1))

    #calculate f_scores against()
    pred_labels_val = classify(images_val, MLP_model, (w0, b0, w1, b1))
    score = performance(pred_labels_val, labels_val)
    best_acc = max(best_acc, score[3])
    # calculate the chage in validation loss
    change = numpy.abs((val_loss-old_val_loss)/old_val_loss)

    # update the counter and old_val_loss
    i += 1
    old_val_loss = val_loss
    
    # print the progress every 10 steps
    if i % 10 == 0:
        print("{}...".format(i), end="")

score = performance(pred_labels_val, labels_val)
print("")
print("")
print("Upon optimization stopped:")
print("    Iterations:", i)
print("    Best Accuracy:", best_acc)

logisitc regression's accuracy: 84%

MLP's accuarcy: 87%

결과적으로 MLP의 성능이 더 좋다. layer의 개수가 많아지고 사이에 non-linear function이 추가되면, 더 다양하고 복잡한 함수를 모델링 할 수 있게 된다. 

그 결과, 데이터를 더 잘 설명할 수 있게 되고, 높은 accuracy를 가질 수 있게 된다. 

